In [2]:
!git clone https://github.com/vllm-project/vllm.git
!cd /content/vllm && VLLM_USE_PRECOMPILED=1 pip install --editable .

!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

Cloning into 'vllm'...
remote: Enumerating objects: 62412, done.
remote: Total 62412 (delta 0), reused 0 (delta 0), pack-reused 62412 (from 1)
Receiving objects: 100% (62412/62412), 40.47 MiB | 16.07 MiB/s, done.
Resolving deltas: 100% (48533/48533), done.
Obtaining file:///content/vllm
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached ninja-1.11.1.3-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using

In [1]:
!pip install -q langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install --upgrade pyzmq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 40.2 MB/s eta 0:00:00
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 24.0.1
    Uninstalling pyzmq-24.0.1:
      Successfully uninstalled pyzmq-24.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.5.5 requires pyzmq<25,>=17, but you have pyzmq 26.3.0 which is incompatible.


In [89]:
!nohup vllm serve google/gemma-3-4b-it --max-model-len 8192  --limit-mm-per-prompt image=1 > gemma.log &

nohup: redirecting stderr to stdout


In [36]:
from transformers import AutoTokenizer, AutoProcessor
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.messages import SystemMessage, HumanMessage

gemma_model_path = 'google/gemma-3-4b-it'
gemma_tokenizer = AutoProcessor.from_pretrained(gemma_model_path)

In [37]:
def gemma_apply_chat_template(prompt_list):
    if isinstance(prompt_list, ChatPromptValue):
        prompt_list = prompt_list.messages
    if isinstance(prompt_list, list):
        formatted_messages = [
            {"role": "system", "content": msg.content} if isinstance(msg, SystemMessage) else
            {"role": "user", "content": msg.content} if isinstance(msg, HumanMessage) else
            {"role": "assistant", "content": msg.content}
            for msg in prompt_list
        ]
        tokenize = gemma_tokenizer.apply_chat_template(
            formatted_messages,
            tokenize=False,
            add_special_tokens=False,
            add_generation_prompt=True
        )
        return tokenize
    else:
        return prompt_list


In [7]:
from langchain_community.llms import VLLMOpenAI

model = VLLMOpenAI(
    openai_api_key="token-abc123",
    openai_api_base="http://0.0.0.0:8000/v1",
    model_name="google/gemma-3-4b-it",
    # temperature=1,
    # top_p=0.95,
    # max_tokens=8192,
    # # frequency_penalty= 0.1,
    # model_kwargs= {"stop": ["<end_of_turn>"]}
)

llm =  gemma_apply_chat_template | model

In [8]:
llm.invoke([HumanMessage(content="안녕하세요.")])

'안녕하세요! 무엇을 도와드릴까요? 궁금한 점이나 필요한 정보가 있으시면 언제든지 말씀해주세요. 😊\n'

In [78]:
def encode_image_base64(image_path):
  with open(image_path, "rb") as f:
    encoded_image = base64.b64encode(f.read())
  encoded_image_text = encoded_image.decode("utf-8")
  base64_qwen = f"data:image;base64,{encoded_image_text}"
  return base64_qwen


In [90]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image_url","image_url": {"url": encode_image_base64('/content/maxresdefault.jpg')}},
            {"type": "text", "text": "이건 뭐야? 한글로 설명해줘."}
        ]
    }
]

In [85]:
model.invoke(messages)

BadRequestError: Error code: 400 - {'object': 'error', 'message': "This model's maximum context length is 8192 tokens. However, you requested 60907 tokens (60651 in the messages, 256 in the completion). Please reduce the length of the messages or completion.", 'type': 'BadRequestError', 'param': None, 'code': 400}

In [91]:
inputs = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt")
inputs

{'input_ids': tensor([[     2,    105,   2364,    107, 237077, 238768, 163694, 236881,  10601,
         239723, 237323,  59587, 237578, 242332, 236761,    106,    107,    105,
           4368,    107]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [92]:
inputs = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False, return_dict=True, return_tensors="pt")
inputs

'<bos><start_of_turn>user\n이건 뭐야? 한글로 설명해줘.<end_of_turn>\n<start_of_turn>model\n'

In [93]:
import base64
from openai import OpenAI


client = OpenAI(
api_key="token-abc123",
base_url="http://0.0.0.0:8000/v1"
)

chat_response = client.chat.completions.create(
    temperature=0.5,
    model="google/gemma-3-4b-it",
    messages=messages,
    max_tokens=512,
)

In [94]:
chat_response.choices[0].message.content

'이 그림은 모찌 (Mochi)라는 일본 캐릭터입니다. \n\n*   **모찌:** 귀여운 디자인으로 유명하며, 주로 찹쌀떡을 연상시키는 둥근 모양을 하고 있습니다.\n*   **특징:** \n    *   하얀색 몸통에 파란색 넥타이를 하고 있습니다.\n    *   눈 주위에 빨간 점이 있고, 콧등에는 옅은 미소를 짓고 있습니다.\n    *   카메라를 들고 사진을 찍는 모습입니다.\n    *   배경에는 바람결을 표현한 선들이 그려져 있습니다.\n\n모찌는 일본에서 매우 인기 있는 캐릭터이며, 다양한 상품으로 판매되고 있습니다.'